In [ ]:
# import cv2
# import numpy as np


# def thinning(binary_image):
#     kernel = np.ones((3, 3), np.uint8)
#     eroded = cv2.erode(binary_image, kernel, iterations=1)
#     result = cv2.subtract(binary_image, eroded)
#     return result


# # Load binary image and convert to single channel grayscale
# binary_image = cv2.imread("road_dil_grey.png", 0)

# # Perform thinning iterations until no change occurs
# prev = None
# while np.all(prev != binary_image):
#     prev = binary_image
#     binary_image = thinning(binary_image)

# cv2.imwrite("thinned_image.png", binary_image)

# ---------------------------------------------------

import cv2
import numpy as np


def skeletonize(binary_image):
    size = np.size(binary_image)
    skel = np.zeros(binary_image.shape, dtype=np.uint8)

    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    done = False

    while not done:
        eroded = cv2.morphologyEx(binary_image, cv2.MORPH_ERODE, element)
        temp = cv2.morphologyEx(eroded, cv2.MORPH_DILATE, element)
        temp = cv2.subtract(binary_image, temp)
        skel = cv2.bitwise_or(skel, temp)
        binary_image = eroded.copy()

        zeros = size - cv2.countNonZero(binary_image)
        if zeros == size:
            done = True

    return skel


# Load binary image and convert to single channel grayscale
binary_image = cv2.imread("road_dil_grey.png", 0)

# Perform skeletonization
skeleton = skeletonize(binary_image)

cv2.imwrite("skeletonized_image.png", skeleton)



In [ ]:
import numpy as np
import cv2


def generate_graph(image):
    rows, cols = image.shape
    graph = {}
    for i in range(rows):
        for j in range(cols):
            if image[i][j] == 255:  # If the pixel is white
                neighbors = []
                if i > 0 and image[i-1][j] == 255:  # Check north
                    neighbors.append((i-1, j))
                if i < rows-1 and image[i+1][j] == 255:  # Check south
                    neighbors.append((i+1, j))
                if j > 0 and image[i][j-1] == 255:  # Check west
                    neighbors.append((i, j-1))
                if j < cols-1 and image[i][j+1] == 255:  # Check east
                    neighbors.append((i, j+1))
                graph[(i, j)] = neighbors
    return graph


binary_image = cv2.imread("skeletonized_image.png", cv2.IMREAD_GRAYSCALE)
graph = generate_graph(binary_image) # DID NOT WORK PROPERLY because nodes are not connected. lines were broken

# plot_graph using the generated graph
binary_image = np.array(binary_image, dtype=np.uint8)

# create a empty image with the same dimensions of binary_image
empty_image = np.zeros(binary_image.shape, dtype=np.uint8)


for node, neighbors in graph.items():
    for neighbor in neighbors:
        i1, j1 = node
        i2, j2 = neighbor
        cv2.line(empty_image, (j1, i1), (j2, i2), (255, 0, 0), 1)
cv2.imwrite("skres.png", empty_image)
